# IMAGINE model library demo

In this short demo, we illustrate the interface of the model library. 
We will discuss 


In [1]:
import ImagineModels as img
import numpy as np

### 1.) Model initialization and evaluation at position

All models are implemented as classes with a similar interface. 
The simplest way to initialize such class (in this case the [Jaffe et al.](https://arxiv.org/abs/0907.3994) model) is via

In [2]:
jaffe = img.JaffeMagneticField()

All models assume a Galacto-centric coordinate system in units of kiloparsec.
They can be evaluated at a position via the `at_position` member function:

In [3]:
position = [2, -2, 0.1]
bfield_at_position = jaffe.at_position(*position)
print(bfield_at_position)

[1.1577395759732363, -0.8220890547803285, 0.0]


### 2.) Evaluation on grid

Alternatively, one may evaluate the model on a regular grid via the `on_grid` member function.
For this, one needs to specify the number of cells, the cell increment and the zeropoint (i.e. the location of the point with minimal coordinate in all directions, again in Galacto-centric coordinates). 
The output will  

In [4]:
grid_shape = [20, 30, 40] # necessarily a list of 3 ints! 
increment = [.2, .3, .01]  
zeropoint = [-10, -5, -.2]

bfield_on_regular_grid = jaffe.on_grid(grid_shape, increment, zeropoint)


The ref to b_grid when filled 0x7ffcb84c3b10 



A third possibility to evaluate the model is on an irregular grid. 
For this, one needs to specify the x, y and z coordinates in separate arrays: 

In [5]:
x_coordinates = [-10, -9.5, 2]
y_coordinates = [-5, -2, 0, .4]
z_coordinates = [-.1, 0, .1]

bfield_on_irregular_grid = jaffe.on_grid(x_coordinates, y_coordinates, z_coordinates)

The ref to b_grid when filled 0x7ffcb84c3b10 



### 3.) Model parameters 

The model parameters are implemented as attributes of the classes:

In [6]:
print(jaffe.disk_amp)

0.167


One can simply update the model parameters via  

In [7]:
jaffe.disk_amp = 3.14
print(jaffe.disk_amp)

3.14


## 4.) Alternative initialization

It is also possible to initialize the models directly with the grid parameters, either regular or irregular: 

In [8]:
jaffe_regular = img.JaffeMagneticField(grid_shape, increment, zeropoint)
jaffe_irregular = img.JaffeMagneticField(x_coordinates, y_coordinates, z_coordinates)

The grid can then be evaluated via 

In [9]:
bfield_on_regular_grid = jaffe_regular.on_grid()

The ref to b_grid when filled 0x7ffcb84c3b70 

on grid f size 3


Other grids can still be passed directly to `on_grid`. 

In the case of the Jaffe model, there is no immediate advantage of passing the grid to the member function or initializing the model with it, apart from maybe more structured code in the latter case. 

For some models, however, initializing the model with the grid maybe advantageous, as some components may be precomputed, resulting in a potentially significant speed advantage.   

### 5.) Random models

Random models (i.e. models which have many degrees of freedom which are not directly accessible, but are drawn from some probability distribution) have the same interface as regular models with some restrictions: 

1. Irregular grids are not supported, due to the fact that the models make use of Fast Fourier Transforms (FFTs).
2. Evaluation at a single position is (not yet) supported.

This only leaves the possibility to evaluate these models on a regular grid. Since random models rely on the FFTW3 module, 
initializing the models with the grid may result in somewhat faster evaluation, as this allows the precomputation of fftw `wisdom'. 
Of course this implies longer initialization.

In [10]:
import time
random_seed = 23

jf12_random_1 = img.JF12RandomField() # intialize without grid

start = time.time()
jf12_random_1.on_grid(grid_shape, increment, zeropoint, random_seed)
end = time.time()
print('Initialized without grid: ',  end - start)

jf12_random_2 = img.JF12RandomField(grid_shape, increment, zeropoint) # intialize with grid
start = time.time()
jf12_random_2.on_grid(random_seed)
end = time.time()
print('Initialized with grid: ',  end - start)


Initialized without grid:  0.007459878921508789
Initialized with grid:  0.004324674606323242
